In [2]:
# https://github.com/maxlath/wikibase-dump-filter
# https://github.com/maxlath/wikibase-dump-filter/blob/main/docs/cli.md
# extracter instance of (P31) city (Q515) eller state in US (Q35657) og som er i country (P17) USA (Q30) eller Mexico (Q96) eller Canada(Q16)
!cat latest-all.json.gz | gzip -d | wikibase-dump-filter --languages en --claim 'P31:Q515,Q35657&P17:Q30,Q96,Q16' --omit sitelinks > city_state_america.ndjson

# extracter instance of (P31) business (Q4830453)
# led efter has subsidiary (P355)
!cat latest-all.json.gz | gzip -d | wikibase-dump-filter --languages en --claim 'P31:Q4830453' --omit sitelinks > business.ndjson


cat: latest-all.json.gz: No such file or directory

gzip: stdin: unexpected end of file
    parsed | total average parse time | recent average parse time |       kept | % of total |   last kept | last kept time | elapsed time
         0 |                      0ms |                       0ms |          0 |         0% |             |              0 |     00:00:01cat: latest-all.json.gz: No such file or directory

gzip: stdin: unexpected end of file
    parsed | total average parse time | recent average parse time |       kept | % of total |   last kept | last kept time | elapsed time
         0 |                      0ms |                       0ms |          0 |         0% |             |              0 |     00:00:01

In [3]:
# https://linuxhint.com/bash_head_tail_command/
!head -n 1 simplified_dump.ndjson > simpl.ndjson

head: cannot open 'simplified_dump.ndjson' for reading: No such file or directory


In [4]:
# https://stedolan.github.io/jq/tutorial/
# https://www.linode.com/docs/guides/using-jq-to-process-json-on-the-command-line/
!jq '.' simpl.ndjson > simpl.json

In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from UtilityFunctions.get_data_path import get_path
import sys
import requests
from math import ceil

In [100]:
def wikidata_query(sparql_query: str):
    # From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent=user_agent)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    return results_df
query_all_us_states = """
SELECT ?state
WHERE {
  ?state wdt:P31 wd:Q35657 .
}
"""
wikidata_query(sparql_query=query_all_us_states)

,state.type,state.value
0,uri,http://www.wikidata.org/entity/Q99
1,uri,http://www.wikidata.org/entity/Q173
2,uri,http://www.wikidata.org/entity/Q724
3,uri,http://www.wikidata.org/entity/Q759
4,uri,http://www.wikidata.org/entity/Q771
5,uri,http://www.wikidata.org/entity/Q779
6,uri,http://www.wikidata.org/entity/Q782
7,uri,http://www.wikidata.org/entity/Q797
8,uri,http://www.wikidata.org/entity/Q812
9,uri,http://www.wikidata.org/entity/Q816


In [128]:
def wikidata_query_categories(category: str):
    # From https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Cats
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent=user_agent)
    sparql.setQuery(f"""
                    SELECT distinct ?item ?itemLabel ?itemDescription WHERE{{  
                    ?item ?label "{category}"@en.  
                    ?article schema:about ?item .
                    ?article schema:inLanguage "en" .
                    ?article schema:isPartOf <https://en.wikipedia.org/>. 
                    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}    
                    }}
                    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    return results_df

In [7]:
wikidata_query_categories('pawn shop')[['item.value', 'itemLabel.value', 'itemDescription.value']].head()

,item.value,itemLabel.value,itemDescription.value
0,http://www.wikidata.org/entity/Q176637,pawnbroker,individual or business that offers loans to pe...


In [131]:
wikidata_query_categories('garden')['item.value'][0][31:]

'Q909444'

In [9]:
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)

schema = pd.read_csv(get_path("schemaorg-current-https-types.csv"))

In [10]:
categories = list(set(biz['categories'].str.cat(sep=', ').split(sep=', ')))

In [11]:
categories

['Airsoft',
 'Teppanyaki',
 'Periodontists',
 'Event Planning & Services',
 'Generator Installation/Repair',
 'Golf Cart Dealers',
 'Amateur Sports Teams',
 'Machine & Tool Rental',
 '3D Printing',
 'Attraction Farms',
 'Cupcakes',
 'Audiologist',
 'Drones',
 'Telecommunications',
 'Drive-Thru Bars',
 'Lounges',
 'Malaysian',
 'Mags',
 'Stonemasons',
 'Gemstones & Minerals',
 'Concierge Medicine',
 'Consumer Law',
 'Vehicle Shipping',
 'Aviation Services',
 'Bird Shops',
 'University Housing',
 'Storefront Clinics',
 'Airports',
 'Tutoring Centers',
 'Badminton',
 'Pawn Shops',
 'Knitting Supplies',
 'Diagnostic Services',
 'Irish Pub',
 'Vietnamese',
 '& Probates',
 'Chinese Martial Arts',
 'Bicycles',
 'Trusts',
 'DUI Schools',
 'Hypnosis/Hypnotherapy',
 'Gymnastics',
 'Educational Services',
 'Traditional Chinese Medicine',
 'Movers',
 'Farmers Market',
 'Urgent Care',
 'Greek',
 'Japanese',
 'Cucina campana',
 'Bike Repair',
 'Fireworks',
 'Boxing',
 'General Festivals',
 'Pop-up S

In [92]:
word = 'Heating & Air Conditioning/HVAC'

def split_word_inc_slash(word):
    # Splitting the words that have a slash in them, and turning them into two words
    word_space = word.split(' ')
    word_space
    new_wordlist_a = []
    new_wordlist_b = []
    for i in word_space:
        i = i.lower()
        if '/' not in i:
            new_wordlist_a.append(i)
            new_wordlist_b.append(i)
        else:
            slash_split = i.split('/')
            new_wordlist_a.append(slash_split[0])
            new_wordlist_b.append(slash_split[1])
    new_word_a = ' '.join(new_wordlist_a)
    new_word_b = ' '.join(new_wordlist_b)
    return [new_word_a, new_word_b]

split_word_inc_slash(word)

['heating & air conditioning', 'heating & air hvac']

In [121]:
# transforms Yelp category words to words that is succesfull in finding the QID on wikidata
# TODO: fix & split issues
# TODO: fix & split has to be done first, and / split after
import inflect
p = inflect.engine()

categories_dict = {}
for word in list(categories):
    if '&' in word:
        word_list = list(filter(None, word.lower().split(sep=' & ')))
        categories_dict[word] = word_list
    if '/' in word:
        categories_dict[word] = split_word_inc_slash(word)
    
categories_dict
categories_dict_singular = {}
for key, value in categories_dict.items():
    new_value = []
    for word in value:
        if p.singular_noun(word) is False:
            word = word
        else:
            word = p.singular_noun(word)
        new_value.append(word)
    categories_dict_singular[key] = new_value

In [122]:
categories_dict_singular

{'Event Planning & Services': ['event planning', 'service'],
 'Generator Installation/Repair': ['generator installation',
  'generator repair'],
 'Machine & Tool Rental': ['machine', 'tool rental'],
 'Gemstones & Minerals': ['gemstone', 'mineral'],
 '& Probates': ['& probate'],
 'Hypnosis/Hypnotherapy': ['hypnosi', 'hypnotherapy'],
 'Bike Repair/Maintenance': ['bike repair', 'bike maintenance'],
 'Recording & Rehearsal Studios': ['recording', 'rehearsal studio'],
 'Keys & Locksmiths': ['key', 'locksmith'],
 'Junk Removal & Hauling': ['junk removal', 'hauling'],
 'Vintage & Consignment': ['vintage', 'consignment'],
 'Kitchen & Bath': ['kitchen', 'bath'],
 'Drywall Installation & Repair': ['drywall installation', 'repair'],
 'Masonry/Concrete': ['masonry', 'concrete'],
 'Public Services & Government': ['public service', 'government'],
 'Baby Gear & Furniture': ['baby gear', 'furniture'],
 'Heating & Air Conditioning/HVAC': ['heating & air conditioning',
  'heating & air hvac'],
 'IP & In

In [123]:
list(categories)[:50]

['Airsoft',
 'Teppanyaki',
 'Periodontists',
 'Event Planning & Services',
 'Generator Installation/Repair',
 'Golf Cart Dealers',
 'Amateur Sports Teams',
 'Machine & Tool Rental',
 '3D Printing',
 'Attraction Farms',
 'Cupcakes',
 'Audiologist',
 'Drones',
 'Telecommunications',
 'Drive-Thru Bars',
 'Lounges',
 'Malaysian',
 'Mags',
 'Stonemasons',
 'Gemstones & Minerals',
 'Concierge Medicine',
 'Consumer Law',
 'Vehicle Shipping',
 'Aviation Services',
 'Bird Shops',
 'University Housing',
 'Storefront Clinics',
 'Airports',
 'Tutoring Centers',
 'Badminton',
 'Pawn Shops',
 'Knitting Supplies',
 'Diagnostic Services',
 'Irish Pub',
 'Vietnamese',
 '& Probates',
 'Chinese Martial Arts',
 'Bicycles',
 'Trusts',
 'DUI Schools',
 'Hypnosis/Hypnotherapy',
 'Gymnastics',
 'Educational Services',
 'Traditional Chinese Medicine',
 'Movers',
 'Farmers Market',
 'Urgent Care',
 'Greek',
 'Japanese',
 'Cucina campana']

In [13]:
category_qid = {}
for i in list(categories)[:500]:
    try:
        cat = i.lower()
        cat_qid = wikidata_query_categories(cat)['item.value'][0][31:]
        category_qid[cat] = cat_qid
    except:
        pass

In [135]:
category_qid_nested = {}
for key, value in categories_dict_singular.items():
    cat_label_qid = {}
    for word in value:
        try:
            cat_qid = wikidata_query_categories(word)['item.value'][0][31:]
        except:
            cat_qid = None
        cat_label_qid[word] = cat_qid
    category_qid_nested[key] = cat_label_qid
category_qid_nested

{'Event Planning & Services': {'event planning': 'Q2040532',
  'service': 'Q906066'},
 'Generator Installation/Repair': {'generator installation': None,
  'generator repair': None},
 'Machine & Tool Rental': {'machine': 'Q11019', 'tool rental': None},
 'Gemstones & Minerals': {'gemstone': 'Q1501187', 'mineral': 'Q7946'},
 '& Probates': {'& probate': None},
 'Hypnosis/Hypnotherapy': {'hypnosi': None, 'hypnotherapy': 'Q1121481'},
 'Bike Repair/Maintenance': {'bike repair': None, 'bike maintenance': None},
 'Recording & Rehearsal Studios': {'recording': 'Q5928097',
  'rehearsal studio': None},
 'Keys & Locksmiths': {'key': 'Q132041', 'locksmith': None},
 'Junk Removal & Hauling': {'junk removal': None, 'hauling': 'Q4174178'},
 'Vintage & Consignment': {'vintage': 'Q1975981', 'consignment': 'Q1368743'},
 'Kitchen & Bath': {'kitchen': 'Q43164', 'bath': 'Q152095'},
 'Drywall Installation & Repair': {'drywall installation': None,
  'repair': None},
 'Masonry/Concrete': {'masonry': 'Q272999', 

In [148]:
category_qid_list = []
for i in category_qid_nested.values():
    for qid in i.values(): 
        if qid is not None:
            category_qid_list.append(qid)
category_qid_list

['Q2040532',
 'Q906066',
 'Q11019',
 'Q1501187',
 'Q7946',
 'Q1121481',
 'Q5928097',
 'Q132041',
 'Q4174178',
 'Q1975981',
 'Q1368743',
 'Q43164',
 'Q152095',
 'Q272999',
 'Q22657',
 'Q11771944',
 'Q7188',
 'Q14745',
 'Q42177',
 'Q80973',
 'Q3046346',
 'Q1144337',
 'Q7361703',
 'Q1936429',
 'Q11020',
 'Q7873',
 'Q21707519',
 'Q18631385',
 'Q731615',
 'Q6686719',
 'Q25221964',
 'Q906066',
 'Q8486',
 'Q22687',
 'Q5122692',
 'Q621580',
 'Q1455871',
 'Q364005',
 'Q34956',
 'Q645858',
 'Q30672236',
 'Q322563',
 'Q160020',
 'Q10874',
 'Q13290',
 'Q192891',
 'Q521839',
 'Q521839',
 'Q3308829',
 'Q7534840',
 'Q48710',
 'Q636641',
 'Q40357',
 'Q36102',
 'Q638',
 'Q34508',
 'Q5994',
 'Q37226',
 'Q7278',
 'Q2040532',
 'Q7600356',
 'Q55633787',
 'Q36963',
 'Q7434376',
 'Q10273457',
 'Q3046346',
 'Q152',
 'Q1074500',
 'Q12796',
 'Q185785',
 'Q652122',
 'Q207116',
 'Q8341',
 'Q1088',
 'Q60061622',
 'Q7010468',
 'Q4390239',
 'Q21071148',
 'Q1285789',
 'Q542869',
 'Q35197',
 'Q5442977',
 'Q5526978',
 

In [151]:
def retrieve_properties_piped(item_list):
    """
    Sends a request to the Wikidata API and transform the data from JSON into a dictionary to
    extract the claims each property has.
    :param item_list: A list with up to 50 wikidata items written with Q-code
    :return: A nested list, with all the properties each item has
    """
    # The limit is set to meet the requirements of the wikibase API wbgetentities (max 50)
    # Ceil makes sure that the each subset from item_list is no longer than 50
    item_list_len = len(item_list)
    limit = ceil(item_list_len / 50)

    # Seperates the item_list to a nested_list with max 50 items in each list
    piped_list = [item_list[pipe::limit] for pipe in range(limit)]
    # Creates the query by seperating each item with "|"
    item_list_query = ""
    for item_list in piped_list:
        for item in range(len(item_list)):
            if item == (len(item_list) - 1):
                item_list_query += item_list[item]
            else:
                item_list_query += item_list[item] + "%7C"

        # The string with API wbgetentities to find multiple items in an optimal format
        URL = "https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids=%s&props=claims&languages=en&formatversion=2" % (
            item_list_query)

        # Opens a HTMl session and gets the DATA from the API
        with requests.Session() as S:
            DATA = dict(S.post(url=URL, headers={"user-agent": "magic browser", "Content-Type": "application/json"}).json())
        # Appends the properties of each item to a nested list
            
        nested_dict = {}
        for entity in DATA["entities"]:
            try:
                nested_dict[entity] = list(DATA["entities"][entity]["claims"].values())
            except:
                pass

    return nested_dict

In [152]:
category_wikidata = retrieve_properties_piped(category_qid_list[:50])

In [153]:
category_triple = {}
for key, values in category_wikidata.items():
    # category_triple_temp_list = []
    # category_triple_temp = {}
    for value in values:
        for obj in value:
            if obj['mainsnak']['property'] == 'P279':
                property = obj['mainsnak']['property']
                data_value = obj['mainsnak']['datavalue']['value']['id']
                category_triple[key] = category_triple.get(key, []) + [{property: data_value}]
category_triple

{'Q2040532': [{'P279': 'Q309100'}],
 'Q906066': [{'P279': 'Q114080183'}, {'P279': 'Q1539016'}],
 'Q11019': [{'P279': 'Q1183543'}, {'P279': 'Q386724'}],
 'Q1501187': [{'P279': 'Q12822126'}],
 'Q7946': [{'P279': 'Q79529'},
  {'P279': 'Q11438'},
  {'P279': 'Q12037703'},
  {'P279': 'Q214609'},
  {'P279': 'Q188460'}],
 'Q1121481': [{'P279': 'Q183257'}],
 'Q132041': [{'P279': 'Q39546'}],
 'Q4174178': [{'P279': 'Q7590'}],
 'Q1975981': [{'P279': 'Q205892'}],
 'Q1368743': [{'P279': 'Q194189'}],
 'Q43164': [{'P279': 'Q180516'}],
 'Q152095': [{'P279': 'Q987767'}, {'P279': 'Q2024731'}],
 'Q272999': [{'P279': 'Q150737'}, {'P279': 'Q214609'}],
 'Q22657': [{'P279': 'Q181790'}, {'P279': 'Q34669510'}],
 'Q11771944': [{'P279': 'Q43229'}],
 'Q7188': [{'P279': 'Q2267705'}, {'P279': 'Q43229'}],
 'Q14745': [{'P279': 'Q31807746'}, {'P279': 'Q1357761'}],
 'Q42177': [{'P279': 'Q14745'}, {'P279': 'Q45422604'}],
 'Q80973': [{'P279': 'Q6460735'}],
 'Q3046346': [{'P279': 'Q1151067'}, {'P279': 'Q17334923'}],
 'Q736